# Challenge 1 - T-test

In statistics, t-test is used to test if two data samples have a significant difference between their means. There are two types of t-test:

* **Student's t-test** (a.k.a. independent or uncorrelated t-test). This type of t-test is to compare the samples of **two independent populations** (e.g. test scores of students in two different classes). `scipy` provides the [`ttest_ind`](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.ttest_ind.html) method to conduct student's t-test.

* **Paired t-test** (a.k.a. dependent or correlated t-test). This type of t-test is to compare the samples of **the same population** (e.g. scores of different tests of students in the same class). `scipy` provides the [`ttest_re`](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.ttest_rel.html) method to conduct paired t-test.

Both types of t-tests return a number which is called the **p-value**. If p-value is below 0.05, we can confidently declare the null-hypothesis is rejected and the difference is significant. If p-value is between 0.05 and 0.1, we may also declare the null-hypothesis is rejected but we are not highly confident. If p-value is above 0.1 we do not reject the null-hypothesis.

Read more about the t-test in [this article](https://researchbasics.education.uconn.edu/t-test/) and [this Quora](https://www.quora.com/What-is-the-difference-between-a-paired-and-unpaired-t-test). Make sure you understand when to use which type of t-test. 

In [19]:
import pandas as pd
import numpy as np

from scipy.stats import ttest_ind
from scipy.stats import ttest_rel
from scipy.stats import f_oneway, norm, f, chi2, chi2_contingency

import warnings
warnings.simplefilter('ignore')

#### Import dataset

In this challenge we will work on the Pokemon dataset you have used last week. The goal is to test whether different groups of pokemon (e.g. Legendary vs Normal, Generation 1 vs 2, single-type vs dual-type) have different stats (e.g. HP, Attack, Defense, etc.).

In [5]:
# Import dataset

pokemon = pd.read_csv('Pokemon.csv')

pokemon.columns = [i.lower().replace(' ','_') for i in pokemon.columns]

pokemon.head()

,#,name,type_1,type_2,total,hp,attack,defense,sp._atk,sp._def,speed,generation,legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


#### First we want to define a function with which we can test the means of a feature set of two samples. 

In the next cell you'll see the annotations of the Python function that explains what this function does and its arguments and returned value. This type of annotation is called **docstring** which is a convention used among Python developers. The docstring convention allows developers to write consistent tech documentations for their codes so that others can read. It also allows some websites to automatically parse the docstrings and display user-friendly documentations.

Follow the specifications of the docstring and complete the function.

In [23]:
def t_test_features(s1, s2, features=['hp', 'attack', 'defense', 'sp._atk', 'sp._def', 'speed', 'total']):
    
    results = {}


    for i in features:

        t_stat, p_value = ttest_ind(s1[i], s2[i])
        results[i] = p_value
    
    return results

In [24]:
legen = pokemon[pokemon.legendary == True]
legen_no = pokemon[pokemon.legendary == False]

In [25]:
t_test_features(legen_no,legen, features=['hp', 'attack', 'defense', 'sp._atk', 'sp._def', 'speed', 'total'])

{'hp': 3.330647684846191e-15,
 'attack': 7.827253003205333e-24,
 'defense': 1.5842226094427255e-12,
 'sp._atk': 6.314915770427266e-41,
 'sp._def': 1.8439809580409594e-26,
 'speed': 2.3540754436898437e-21,
 'total': 3.0952457469652825e-52}

#### From the test results above, what conclusion can you make? Do Legendary and non-Legendary pokemons have significantly different stats on each feature?

Si, en todos los estadísticos p_value es menor que 0.05, por tanto podemos rechazar H0 y determinar que los valores son diferentes para los pokemon normales y los legendarios.

#### Next, conduct t-test for Generation 1 and Generation 2 pokemons.

In [26]:
t_test_features(pokemon[pokemon.generation == 1],pokemon[pokemon.generation == 2], features=['hp', 'attack', 'defense', 'sp._atk', 'sp._def', 'speed', 'total'])

{'hp': 0.13791881412813622,
 'attack': 0.24050968418101457,
 'defense': 0.5407630349194362,
 'sp._atk': 0.14119788176331508,
 'sp._def': 0.16781226231606386,
 'speed': 0.0028356954812578704,
 'total': 0.5599140649014442}

#### What conclusions can you make?

En el caso de las generaciones 1 y 2 no podemos rechazar la hipótesis nula excepto en el caso del atributo `speed`. Solo hay diferencia significativa en speed.

#### Compare pokemons who have single type vs those having two types.

In [34]:
one = pokemon[pd.isna(pokemon['type_2'])]
multi = pokemon[~pd.isna(pokemon['type_2'])]
t_test_features(one,
                multi,
                features=['hp', 'attack', 'defense', 'sp._atk', 'sp._def', 'speed', 'total'])

{'hp': 0.11060643144431842,
 'attack': 0.00015741395666164396,
 'defense': 3.250594205757004e-08,
 'sp._atk': 0.0001454917404035147,
 'sp._def': 0.00010893304795534396,
 'speed': 0.024051410794037463,
 'total': 1.1749035008828752e-07}

#### What conclusions can you make?

Vemos que en el caso de pokemon con uno o dos tipos podemos rechazar la hipótesis nula en varios atributos, entre ellos: `[attack, defense, special_attack, special_defense, speed, total]`. Únicamente no podemos rechazar la hipótesis nula en el atributo `HP`, en este es en el único en el que existen diferencias significativas.

#### Now, we want to compare whether there are significant differences of `Attack` vs `Defense`  and  `Sp. Atk` vs `Sp. Def` of all pokemons. Please write your code below.

*Hint: are you comparing different populations or the same population?*

In [54]:
pokemon.columns = [i.replace('.','_') for i in pokemon.columns]
pokemon.head()

,#,name,type_1,type_2,total,hp,attack,defense,sp__atk,sp__def,speed,generation,legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [50]:
from statsmodels.multivariate.manova import MANOVA

In [95]:
formula = 'attack + defense ~ sp__atk + sp__def - 1'
# Comparación de los atributos sin considerar ninguna variable dependiente

In [96]:
manova = MANOVA.from_formula(formula, pokemon)

manova

In [97]:
print(manova.mv_test())

                  Multivariate linear model
                                                              
--------------------------------------------------------------
        sp__atk         Value  Num DF  Den DF  F Value  Pr > F
--------------------------------------------------------------
          Wilks' lambda 0.7850 2.0000 797.0000 109.1524 0.0000
         Pillai's trace 0.2150 2.0000 797.0000 109.1524 0.0000
 Hotelling-Lawley trace 0.2739 2.0000 797.0000 109.1524 0.0000
    Roy's greatest root 0.2739 2.0000 797.0000 109.1524 0.0000
--------------------------------------------------------------
                                                              
--------------------------------------------------------------
        sp__def         Value  Num DF  Den DF  F Value  Pr > F
--------------------------------------------------------------
          Wilks' lambda 0.5669 2.0000 797.0000 304.4494 0.0000
         Pillai's trace 0.4331 2.0000 797.0000 304.4494 0.0000
 Hotelling-

#### What conclusions can you make?

In [ ]:
# Your comment here